# Подготовка файлов и программ

Некоторые программы ставятся дважды из разных источников для демонстрации различных вариантов в случае неполадок. Для оптимизации времени работы необходимо удалить лишний код.

In [1]:
pip install -q condacolab

In [2]:
import condacolab

In [ ]:
condacolab.install()

In [ ]:
!conda install -c bioconda seqtk

In [ ]:
!conda install -c bioconda trimmomatic

In [ ]:
!pip install macs2 intervene

In [ ]:
!apt-get install bedtools

In [ ]:
!apt-get install bowtie2 samtools

In [ ]:
!conda install -c bioconda bowtie2

## Установка FastQC

In [ ]:
!java -version

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip

In [ ]:
!unzip fastqc_v0.11.9.zip

In [7]:
!chmod a+x FastQC/fastqc

In [ ]:
!./FastQC/fastqc --help

# Выравнивание

## Скачиваем чтения

Для примера чтения были взяты из эксперимента https://www.encodeproject.org/experiments/ENCSR000AKC/

In [ ]:
# ChIP-seq на гистоновой метке
!wget https://www.encodeproject.org/files/ENCFF002AYC/@@download/ENCFF002AYC.fastq.gz
!wget https://www.encodeproject.org/files/ENCFF002AVT/@@download/ENCFF002AVT.fastq.gz
# ChIP-seq контроль
!wget https://www.encodeproject.org/files/ENCFF002BBC/@@download/ENCFF002BBC.fastq.gz

In [10]:
# распаковка ахрива
!gzip -d /content/ENCFF002AYC.fastq.gz /content/ENCFF002AVT.fastq.gz /content/ENCFF002BBC.fastq.gz

## FastQC (или multiQC может быть использован в работе)

In [11]:
!./FastQC/fastqc /content/ENCFF002AYC.fastq
!./FastQC/fastqc /content/ENCFF002AVT.fastq
!./FastQC/fastqc /content/ENCFF002BBC.fastq

Started analysis of ENCFF002AYC.fastq
Approx 5% complete for ENCFF002AYC.fastq
Approx 10% complete for ENCFF002AYC.fastq
Approx 15% complete for ENCFF002AYC.fastq
Approx 20% complete for ENCFF002AYC.fastq
Approx 25% complete for ENCFF002AYC.fastq
Approx 30% complete for ENCFF002AYC.fastq
Approx 35% complete for ENCFF002AYC.fastq
Approx 40% complete for ENCFF002AYC.fastq
Approx 45% complete for ENCFF002AYC.fastq
Approx 50% complete for ENCFF002AYC.fastq
Approx 55% complete for ENCFF002AYC.fastq
Approx 60% complete for ENCFF002AYC.fastq
Approx 65% complete for ENCFF002AYC.fastq
Approx 70% complete for ENCFF002AYC.fastq
Approx 75% complete for ENCFF002AYC.fastq
Approx 80% complete for ENCFF002AYC.fastq
Approx 85% complete for ENCFF002AYC.fastq
Approx 90% complete for ENCFF002AYC.fastq
Approx 95% complete for ENCFF002AYC.fastq
Analysis complete for ENCFF002AYC.fastq
Started analysis of ENCFF002AVT.fastq
Approx 5% complete for ENCFF002AVT.fastq
Approx 10% complete for ENCFF002AVT.fastq
Appr

## Выравнивание на хромосому

Следует выбрать одну хромосому, потому что ресурсы google colab ограничены. В случае проблем, таких как Segmentation fault, error 139, следует выбрать хромосому меньшего размера.

Последовательность нуклеотидов хромосомы можно скачать по адресу https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/

In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr18.fa.gz
!gzip -d chr18.fa.gz

In [13]:
%%time
!bowtie2-build chr18.fa chromosome_index

Settings:
  Output files: "chromosome_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  chr18.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 20022401
Using parameters --bmax 15016801 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 15016801 --dcv 1024
Constructing suffix-arr

In [14]:
!mkdir bowtie2_res

In [15]:
%%time
!bowtie2  -p 2 \
          -x chromosome_index \
          -U ENCFF002AYC.fastq \
          -S bowtie2_res/res_AYC.sam

42224764 reads; of these:
  42224764 (100.00%) were unpaired; of these:
    36548730 (86.56%) aligned 0 times
    1474101 (3.49%) aligned exactly 1 time
    4201933 (9.95%) aligned >1 times
13.44% overall alignment rate
CPU times: user 8.91 s, sys: 1.1 s, total: 10 s
Wall time: 18min 11s


In [16]:
%%time
!bowtie2  -p 2 \
          -x chromosome_index \
          -U ENCFF002AVT.fastq \
          -S bowtie2_res/res_AVT.sam

26657759 reads; of these:
  26657759 (100.00%) were unpaired; of these:
    23683146 (88.84%) aligned 0 times
    795832 (2.99%) aligned exactly 1 time
    2178781 (8.17%) aligned >1 times
11.16% overall alignment rate
CPU times: user 4.72 s, sys: 519 ms, total: 5.24 s
Wall time: 9min 23s


In [17]:
%%time
!bowtie2  -p 2 \
          -x chromosome_index \
          -U ENCFF002BBC.fastq \
          -S bowtie2_res/res_BBC.sam

42527005 reads; of these:
  42527005 (100.00%) were unpaired; of these:
    35889576 (84.39%) aligned 0 times
    1589790 (3.74%) aligned exactly 1 time
    5047639 (11.87%) aligned >1 times
15.61% overall alignment rate
CPU times: user 8.98 s, sys: 1.04 s, total: 10 s
Wall time: 18min 18s


Проанализируйте выдачу bowtie. Почему процент выравниваний получился именно таким?

Имеет смысл для дальнейшего анализа отобрать уникально картированные риды.

Преобразование .sam в .bam (не требуется если запускать macs2 на .sam файлах)

На этом моменте можно скачать себе на компьютер все полученные .bam или .sam файлы на случай сбоя.

## Peak calling

Помимо macs2 может быть использован пакет Homer для поиска пиков, а также macs3.

In [18]:
!mkdir macs2

In [ ]:
!macs2 callpeak --broad -t bowtie2_res/res_AYC.sam \
    -c bowtie2_res/res_BBC.sam \
 	  -f SAM \
	  --outdir macs2_AYC

In [ ]:
!macs2 callpeak --broad -t bowtie2_res/res_AVT.sam \
    -c bowtie2_res/res_BBC.sam \
 	  -f SAM \
	  --outdir macs2_AVT

In [ ]:
!wget https://www.encodeproject.org/files/ENCFF626WXW/@@download/ENCFF626WXW.bed.gz
!gzip -d ENCFF626WXW.bed.gz

## Сравнение результатов

Сравниваем те пики, которые мы получили, с пиками, которые приведены в ENCODE (важно, чтобы версии генома hg38 или hg19) для .bed файла из ENCODE и той хромосомы, которую скачивали выше, сопадали.

Проанализируйте полученные результаты и приведите свои рассуждения в README.md. Как можно объяснить различия в количестве пересечений?

In [22]:
!intervene venn -i macs2_AYC/NA_peaks.broadPeak ENCFF626WXW.bed --filenames --output venn_results/venn1.jpg
!intervene venn -i ENCFF626WXW.bed macs2_AYC/NA_peaks.broadPeak --filenames --output venn_results/venn2.jpg


Generating a 2-way "venn" diagram. Please wait...


Done! Please check your results @ venn_results/venn1.jpg. 
Thank you for using Intervene!


Generating a 2-way "venn" diagram. Please wait...


Done! Please check your results @ venn_results/venn2.jpg. 
Thank you for using Intervene!



In [23]:
!intervene venn -i macs2_AVT/NA_peaks.broadPeak ENCFF626WXW.bed --filenames --output venn_results/venn3.jpg
!intervene venn -i ENCFF626WXW.bed macs2_AVT/NA_peaks.broadPeak --filenames --output venn_results/venn4.jpg


Generating a 2-way "venn" diagram. Please wait...


Done! Please check your results @ venn_results/venn3.jpg. 
Thank you for using Intervene!


Generating a 2-way "venn" diagram. Please wait...


Done! Please check your results @ venn_results/venn4.jpg. 
Thank you for using Intervene!



# Бонусная часть

Задание: получить ngsplot и heatmap (загрузить на гитхаб), проанализировать его, сравнив с теоретической версией локализации гистоновой метки. Как можно объяснить вид распределения и локализацию мод?

Ниже приведён пример для подсчёта ngsplot с использованием опубликованного .bigWig файла, а также приведено возможное получение .bigWig файла.

Проверяйте все файлы на совпадение версии сборки генома, которую вы используете!

Ищем .bam file
https://www.encodeproject.org/experiments/ENCSR000AKC/

Ищем .bigWig file связанный с выбранным .bam
https://www.encodeproject.org/experiments/ENCSR000AKC/

Более подробную инструкцию можно найти в чате в телеграмм (сообщения в 11:30 1 марта)

## Установка deeptools и bedtools

In [24]:
#!pip install imgaug==0.2.5

In [25]:
!pip install -q deeptools

     |████████████████████████████████| 233 kB 31.0 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 27.7 MB 4.2 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 
     |████████████████████████████████| 41 kB 110 kB/s 
     |████████████████████████████████| 3.1 MB 45.1 MB/s 
     |████████████████████████████████| 133 kB 58.4 MB/s 
     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 93 kB 1.2 MB/s 
     |████████████████████████████████| 100 kB 8.2 MB/s 
     |████████████████████████████████| 575 kB 44.3 MB/s 
     |████████████████████████████████| 121 kB 48.5 MB/s 
     |████████████████████████████████| 90 kB 7.3 MB/s 
     |████████████████████████████████| 8.8 MB 42.3 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 1.1 MB 41.9 MB/s 


In [26]:
!conda install -c bioconda bedtools

Solving environment: | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bedtools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bedtools-2.30.0            |       h7d7f7ad_1        17.9 MB  bioconda
    ------------------------------------------------------------
                                           Total:        17.9 MB

The following NEW packages will be INSTALLED:

  bedtools           bioconda/linux-64::bedtools-2.30.0-h7d7f7ad_1



bedtools-2.30.0      | 17.9 MB   | : 100% 1.0/1 [00:03<00:00,  3.42s/it]               
Preparing transaction: | done
Verifying transaction: - \ | / - \ | / done
Executing transaction: \ done


## Подсчёт ngsplot при помощи deeptools
Необходимо скачать аннотацию (например, для hg19 взять отсюда https://hgdownload-test.gi.ucsc.edu/goldenPath/hg19/bigZips/genes/ - hg19.knownGene)
Для hg38 - https://hgdownload-test.gi.ucsc.edu/goldenPath/hg38/bigZips/genes/

In [ ]:
!wget https://hgdownload-test.gi.ucsc.edu/goldenPath/hg38/bigZips/genes/hg38.knownGene.gtf.gz

In [ ]:
!wget https://www.encodeproject.org/files/ENCFF226TQJ/@@download/ENCFF226TQJ.bigWig
!wget https://www.encodeproject.org/files/ENCFF157QMD/@@download/ENCFF157QMD.bigWig

In [ ]:
# подсчёт матрицы. В примере считается 30 минут.
%%time
!computeMatrix scale-regions \
 -S ENCFF226TQJ.bigWig \
 -R hg38.knownGene.gtf.gz \
 -out matrix1.tab.gz \
 -a 2000 -b 2000 \
 --regionBodyLength 4000 \
 --skipZeros \
 --missingDataAsZero -p 2

In [ ]:
# подсчёт матрицы. В примере считается 30 минут.
%%time
!computeMatrix scale-regions \
 -S ENCFF157QMD.bigWig \
 -R hg38.knownGene.gtf.gz \
 -out matrix2.tab.gz \
 -a 2000 -b 2000 \
 --regionBodyLength 4000 \
 --skipZeros \
 --missingDataAsZero -p 2

In [31]:
!plotHeatmap \
 -m matrix1.tab.gz \
 -out result1.png \
 --colorMap YlGnBu \
 --regionsLabel 'genes' \
 --heatmapHeight 15 \
 --plotTitle 'ngs_plot' &

tcmalloc: large alloc 1433444352 bytes == 0x56518c88a000 @  0x7f28c7c1b1e7 0x7f28c5fb10ce 0x7f28c6007cf5 0x7f28c60b086d 0x7f28c60b117f 0x7f28c60b12d0 0x56511fe2cd5f 0x7f28c5ff2944 0x56511feba3eb 0x56511febbad8 0x56511fee24ac 0x56511fe29af2 0x56511fe58030 0x56511febb9c8 0x56511fee674a 0x56511fe2aead 0x7f28c5ff2944 0x56511feba3eb 0x56511febbad8 0x56511fee24ac 0x56511fe29af2 0x56511fe58030 0x56511febb9c8 0x56511fee674a 0x56511fe57e94 0x56511febb9c8 0x56511fee2544 0x56511fe29af2 0x56511fe58030 0x56511febb9c8 0x56511fee24ac
tcmalloc: large alloc 1433444352 bytes == 0x5651f7558000 @  0x7f28c7c1b1e7 0x7f28c5fb10ce 0x7f28c600b726 0x7f28c5ffe475 0x7f28c60ae6ec 0x56511feba300 0x56511febbad8 0x56511fee31d9 0x56511fe57e94 0x56511febb9c8 0x56511fee24ac 0x56511fe29af2 0x56511fe2b3a8 0x7f28c5ff2944 0x56511feba3eb 0x56511febbad8 0x56511fee24ac 0x56511fe29af2 0x56511fe58030 0x56511febb9c8 0x56511fee31d9 0x56511fe29af2 0x56511fe58030 0x56511febb9c8 0x56511fee31d9 0x56511fe29af2 0x56511fe58030 0x56511feb

In [32]:
!plotHeatmap \
 -m matrix2.tab.gz \
 -out result2.png \
 --colorMap YlGnBu \
 --regionsLabel 'genes' \
 --heatmapHeight 15 \
 --plotTitle 'ngs_plot' &

tcmalloc: large alloc 1425596416 bytes == 0x55bb2938e000 @  0x7fa42ec601e7 0x7fa42cff60ce 0x7fa42d04ccf5 0x7fa42d0f586d 0x7fa42d0f617f 0x7fa42d0f62d0 0x55babeb5ad5f 0x7fa42d037944 0x55babebe83eb 0x55babebe9ad8 0x55babec104ac 0x55babeb57af2 0x55babeb86030 0x55babebe99c8 0x55babec1474a 0x55babeb58ead 0x7fa42d037944 0x55babebe83eb 0x55babebe9ad8 0x55babec104ac 0x55babeb57af2 0x55babeb86030 0x55babebe99c8 0x55babec1474a 0x55babeb85e94 0x55babebe99c8 0x55babec10544 0x55babeb57af2 0x55babeb86030 0x55babebe99c8 0x55babec104ac
tcmalloc: large alloc 1425596416 bytes == 0x55bb93700000 @  0x7fa42ec601e7 0x7fa42cff60ce 0x7fa42d050726 0x7fa42d043475 0x7fa42d0f36ec 0x55babebe8300 0x55babebe9ad8 0x55babec111d9 0x55babeb85e94 0x55babebe99c8 0x55babec104ac 0x55babeb57af2 0x55babeb593a8 0x7fa42d037944 0x55babebe83eb 0x55babebe9ad8 0x55babec104ac 0x55babeb57af2 0x55babeb86030 0x55babebe99c8 0x55babec111d9 0x55babeb57af2 0x55babeb86030 0x55babebe99c8 0x55babec111d9 0x55babeb57af2 0x55babeb86030 0x55babebe

Получаем результат в файле "result.png"